In [2]:
from IPython import get_ipython
from IPython.display import display
# %%
# Install library yang diperlukan
!pip install google-play-scraper pandas

import pandas as pd
from google_play_scraper import app, reviews as g_reviews, Sort # Import Sort enum
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import time
from google_play_scraper import reviews as g_reviews, Sort
from datetime import datetime

# --- Konfigurasi Awal ---
app_id = 'jp.naver.line.android'  # ID Aplikasi LINE
# Tentukan batas tanggal akhir (1 Januari tahun ini, atau ganti tahun sesuai kebutuhan)
target_date = datetime(2025, 1, 1)

all_reviews = []

# --- Fungsi Scraping Berdasarkan Tanggal ---
def scrape_google_play_reviews_by_date():
    continuation_token = None
    is_scraping = True

    while is_scraping:
        batch, continuation_token = g_reviews(
            app_id,
            lang='id',
            country='id',
            sort=Sort.NEWEST, # Wajib NEWEST agar urut dari terbaru
            count=200,
            continuation_token=continuation_token
        )

        # Jika tidak ada data yang diambil, hentikan
        if not batch:
            break

        # Filter batch saat ini
        for review in batch:
            review_date = review['at'] # 'at' biasanya sudah dalam format datetime

            # Cek apakah tanggal ulasan masih >= Target (1 Januari)
            if review_date >= target_date:
                all_reviews.append(review)
            else:
                # Jika ketemu ulasan yang lebih tua dari 1 Januari, hentikan loop utama
                is_scraping = False
                break

        print(f"Mengambil data... Total terkumpul: {len(all_reviews)}")

        # Hentikan jika tidak ada token lanjutan
        if not continuation_token:
            break

        time.sleep(2) # Delay untuk menghindari blokir

    return all_reviews

# --- Eksekusi ---
line_reviews = scrape_google_play_reviews_by_date()

# --- Konversi ke DataFrame ---
df = pd.DataFrame(line_reviews)

if not df.empty:
    df = df[['userName', 'content', 'score', 'at']]
    # Simpan ke CSV
    df.to_csv('line_reviews_datatest.csv', index=False)
    print("\nData berhasil disimpan!")
    print(df.head(4))
else:
    print("Tidak ada ulasan ditemukan dalam rentang tanggal tersebut.")

In [4]:
import pandas as pd
import time
from google_play_scraper import reviews as g_reviews, Sort
from datetime import datetime

# --- Konfigurasi Awal ---
app_id = 'jp.naver.line.android'
target_date = datetime(2024, 1, 1)
target_count = 10  # TARGET JUMLAH DATA

all_reviews = []

def scrape_google_play_reviews_limited():
    continuation_token = None
    is_scraping = True

    while is_scraping:
        batch, continuation_token = g_reviews(
            app_id,
            lang='id',
            country='id',
            sort=Sort.NEWEST,
            count=10,
            continuation_token=continuation_token
        )

        if not batch:
            break

        for review in batch:
            review_date = review['at']

            # Ambil ulasan jika tanggalnya masih masuk kriteria (>= 1 Jan 2024)
            if review_date >= target_date:
                all_reviews.append(review)

                # --- MODIFIKASI: Cek apakah sudah mencapai target (10 data) ---
                if len(all_reviews) >= target_count:
                    is_scraping = False
                    break
            else:
                # Jika ulasan sudah lebih tua dari target date, berhenti total
                is_scraping = False
                break

        print(f"Mengambil data... Total terkumpul: {len(all_reviews)}")

        if not continuation_token:
            break

        # Delay hanya jika kita masih lanjut scraping
        if is_scraping:
            time.sleep(2)

    return all_reviews

# --- Eksekusi ---
print("Memulai proses scraping...")
line_reviews = scrape_google_play_reviews_limited()

# --- Konversi ke DataFrame ---
df = pd.DataFrame(line_reviews)

if not df.empty:
    # Filter kolom yang dibutuhkan
    df = df[['userName', 'content', 'score', 'at']]

    # Simpan ke CSV
    filename = 'line_reviews_sample_10.csv'
    df.to_csv(filename, index=False)
    print(f"\nBerhasil! {len(df)} data disimpan ke '{filename}'.")
    print(df.head(10)) # Tampilkan semua hasil (max 10)
else:
    print("Tidak ada ulasan ditemukan.")

Memulai proses scraping...
Mengambil data... Total terkumpul: 10

Berhasil! 10 data disimpan ke 'line_reviews_sample_10.csv'.
               userName                                            content  \
0                   tds  aplikasi ruwet. mau buka akun lama tp lupa pas...   
1     Rahmat hidayat#14  I like this application, it is very satisfying...   
2       Andhika Permana                  sulit buat login ketika pindah hp   
3        Evo Blackcover                                ribet amat log-in '   
4           David Rusli                                              bagus   
5         Rere MixologY  susah sekali mau masuk ke akun harus masukin k...   
6  Kenny Jevon A. Ntiri  saya punya hp Ke restart tidak bisa login ke p...   
7          Raka Tampani  RIBET MAU LOGIN AJA SUSAH GK KEK YANG LAIN CON...   
8         Bisindo Class                                              susah   
9       Dhina qweenrd91  aku suka aplikasih ini sangat membantu bisa sa...   

   score       

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
from google.colab import data_table
from IPython.display import display

# --- 1. Konfigurasi Path (Sesuaikan path ini!) ---
base_model_path = "/content/drive/MyDrive/Dataset/Epoch 2"
input_csv = 'line_reviews_sample_10.csv'
topics = ['Indobert_topic_1', 'Indobert_topic_2', 'Indobert_topic_3']

id2label = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

# --- 2. Fungsi Prediksi ---
def get_prediction(text, tokenizer, model):
    if not isinstance(text, str) or not text.strip():
        return "Empty"

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    model.to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class_id = torch.argmax(outputs.logits, dim=1).item()
    return id2label.get(predicted_class_id, "Unknown")

# --- 3. Eksekusi ---
try:
    print("Membaca data...")
    df = pd.read_csv(input_csv)

    # Loop 3 Model
    for topic in topics:
        full_path = os.path.join(base_model_path, topic)
        print(f"⏳ Memproses {topic}...")

        try:
            tokenizer = AutoTokenizer.from_pretrained(full_path)
            model = AutoModelForSequenceClassification.from_pretrained(full_path)

            # Buat kolom baru untuk setiap topic
            df[f'Sentimen {topic}'] = df['content'].apply(lambda x: get_prediction(x, tokenizer, model))

        except OSError:
            df[f'Sentimen {topic}'] = "Model Not Found"

    # --- 4. TAMPILKAN SEBAGAI TABEL ---
    print("\n" + "="*20 + " HASIL PREDIKSI " + "="*20)

    # Aktifkan fitur tabel interaktif Colab
    data_table.enable_dataframe_formatter()

    # Pilih kolom yang ingin ditampilkan agar rapi
    # Kita ambil content dan 3 kolom hasil prediksi
    cols = ['content'] + [f'Sentimen {t}' for t in topics]

    # Tampilkan tabel (Hanya 10 data sesuai request)
    display(df[cols].head(10))

    # Simpan
    df.to_csv('hasil_banding_3_model.csv', index=False)
    print("\n✅ File CSV tersimpan sebagai 'hasil_banding_3_model.csv'")

except FileNotFoundError:
    print("❌ File CSV input tidak ditemukan.")

Membaca data...
⏳ Memproses Indobert_topic_1...
⏳ Memproses Indobert_topic_2...
⏳ Memproses Indobert_topic_3...

==================== HASIL PREDIKSI ====================


,content,Sentimen Indobert_topic_1,Sentimen Indobert_topic_2,Sentimen Indobert_topic_3
0,aplikasi ruwet. mau buka akun lama tp lupa pas...,Neutral,Neutral,Negative
1,"I like this application, it is very satisfying...",Positive,Neutral,Neutral
2,sulid buad logiyn pas mw bindah hengpon,Neutral,Neutral,Negative
3,"Fitur tambahan LINE sebenarnya bagus, tapi tid...",Neutral,Negative,Neutral
4,bagus,Positive,Neutral,Neutral
5,susah sekali mau masuk ke akun harus masukin k...,Neutral,Neutral,Negative
6,saya punya hp Ke restart tidak bisa login ke p...,Neutral,Neutral,Negative
7,"Fitur tambahan di LINE sekarang makin lengkap,...",Neutral,Negative,Neutral
8,susah,Neutral,Neutral,Neutral
9,aku suka aplikasih ini sangat membantu bisa sa...,Positive,Neutral,Neutral



✅ File CSV tersimpan sebagai 'hasil_banding_3_model.csv'
